In [ ]:
!pip install peft transformers datasets accelerate

In [ ]:
! pip install peft
! pip install datasets accelerate

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import PromptTuningConfig, get_peft_model, TaskType
from datasets import Dataset

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
inputs = tokenizer("Hello world!", return_tensors="pt")
model(**inputs)  # This is the right way


In [ ]:

# 1. Load tokenizer & frozen GPT2 model
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # GPT2 doesn't have a pad token

model = AutoModelForCausalLM.from_pretrained(model_name)

# 2. Create a tiny dataset (you can expand this)
examples = [
    {"text": "The movie was great!", "label": "positive"},
    {"text": "I hated the film.", "label": "negative"},
    {"text": "Absolutely loved it!", "label": "positive"},
    {"text": "It was terrible.", "label": "negative"},
]


In [7]:
def preprocess(example):
    enc = tokenizer(f"Review: {example['text']} Sentiment:", padding="max_length", max_length=32, truncation=True)
    enc["labels"] = enc["input_ids"].copy()  # causal LM expects this
    return enc


In [8]:

dataset = Dataset.from_list(examples).map(preprocess)

# 3. Setup PEFT config for soft prompting
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    num_virtual_tokens=10,
    tokenizer_name_or_path=model_name
)

# 4. Add soft prompt to model (only prompt gets trained)
model = get_peft_model(model, peft_config)

# 5. Training settings (use CPU)
training_args = TrainingArguments(
    output_dir="./soft_prompt_output",
    per_device_train_batch_size=1,
    num_train_epochs=20,
    logging_steps=1,
    save_strategy="no",
    remove_unused_columns=False,
    label_names=["labels"]  # 👈 Explicitly set
)


# 6. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

trainer.train()


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

ValueError: too many dimensions 'str'